In [1]:
# Imports:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import sklearn
from sklearn.metrics import mean_squared_error
import plotly.express as px
from scipy import stats


In [2]:
#set seed for reproducibility
random_state = 42
np.random.seed(random_state)

In [3]:
# same dataset as bidirectional, which is why the path is the same
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_canada_dataset_dict.pickle', 'rb') as f:
    cananda_data_dict = pickle.load(f)

In [4]:
us_data = pd.read_csv(r'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/resources/canada_cpi_dataset_linear_regression_weights.csv')

--------

### Horizon Analyses:

In [6]:
# import horizon files 
with open('//Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - Canada/data/hor1_test_dataset.pickle', 'rb') as f:
    canada_hor_1_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - Canada/data/hor2_test_dataset.pickle', 'rb') as f:
    canada_hor_2_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - Canada/data/hor3_test_dataset.pickle', 'rb') as f:
    canada_hor_3_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - Canada/data/hor4_test_dataset.pickle', 'rb') as f:
    canada_hor_4_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - Canada/data/hor8_test_dataset.pickle', 'rb') as f:
    canada_hor_8_test_dict = pickle.load(f)

In [7]:
canada_hor_1_test_dict['All-items'].shape

(26, 14)

In [8]:
import sys        
sys.path.append('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model_canada/model')       
from GRU_model import *
from config import * 
from utils import *
Lr = 0.09138694552832571

sys.path.append('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model')       
from pipeline_config import *

2023-09-30 22:16:07.485744: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
num_forecast_steps = 8

category_horizon_dict = {}

for category_name in list(canada_hor_1_test_dict.keys()):
    # load model
    cat_model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
    cat_optimizer = torch.optim.AdamW(cat_model.parameters(), lr=Lr)
    cat_model.to(Device)

    category_model, optimizer, checkpoint, valid_loss_min = load_checkpoint('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model_canada/models_weights/'+category_name+'.pt', cat_model, cat_optimizer)

    horizon_dict = {}

    # Generate forecasts step by step
    for step in range(num_forecast_steps):
        with torch.no_grad():
            # create dataloader
            x_test = canada_hor_1_test_dict[category_name].iloc[:,:-1].to_numpy()
            y_test= canada_hor_1_test_dict[category_name].iloc[:,-1:].to_numpy()

            x_test = torch.from_numpy(x_test).to(torch.float32)
            y_test = torch.from_numpy(y_test).to(torch.float32)

            test_dataset = TensorDataset(x_test, y_test)

            test_dataloader =  DataLoader(test_dataset, batch_size=BatchSize, shuffle=False)

            for inputs, labels in test_dataloader:
                # since each batch contains 32 samples, and the largest df is of size 32, then we dont need to concat the outputs per batch per category.
                inputs = inputs.view(inputs.shape[0], SequenceLength, Features)
                #print(f'inputs shape: {inputs.shape}')
                #print(f'labels shape: {labels.shape}')
                inputs, labels = inputs.to(Device), labels.to(Device)
                out = category_model(inputs)
                out_df = pd.DataFrame(out)
                
                first = canada_hor_1_test_dict[category_name].iloc[:,1:].reset_index(drop = True).copy()
                print(f'first shape: {first.shape}')
                second = out_df.reset_index(drop = True).rename(columns = {0: 'Inflation t+'+str(2+step)})
                print(f'second shape: {second.shape}')

                canada_hor_1_test_dict[category_name] = pd.concat([first, second],axis=1) #prediction

                horizon_dict[step+2] = canada_hor_1_test_dict[category_name]

    category_horizon_dict[category_name] = horizon_dict
    


first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second shape: (26, 1)
first shape: (26, 13)
second sha

In [10]:
def avg_rmse_hor(prediction_dict, actual_dict, horizon):
    mse_lst = []
    for key in list(actual_dict.keys()):
        predictions = prediction_dict[key][horizon].iloc[:,-1:].reset_index(drop = True)
        actuals = actual_dict[key].iloc[:,-1:].reset_index(drop = True)

        y_pred = predictions.values
        y_actual = actuals.values

        non_null_indices = ~np.isnan(y_pred) & ~np.isnan(y_actual)

        predicted_values_non_null = y_pred[non_null_indices]
        observed_values_non_null = y_actual[non_null_indices]

        #print(f'y pred: {predicted_values_non_null}')
        #print(f'y actual: {observed_values_non_null}')
        mse = mean_squared_error(predicted_values_non_null, observed_values_non_null)
        mse_lst.append(mse)
    
    rmse_list = list(map(np.sqrt,mse_lst))
    avg_rmse = np.mean(rmse_list)
    rmse_std = np.std(rmse_list)
    
    print(f'RMSE:  {avg_rmse}')
    print(f'MSE std:  {rmse_std}')
    print(f'interval: {[avg_rmse-rmse_std, avg_rmse+rmse_std]}')

    return avg_rmse,rmse_std

In [11]:
avg_rmse_hor(category_horizon_dict, canada_hor_2_test_dict, 2)

RMSE:  2.2503864389111965
MSE std:  1.9644135033449535
interval: [0.28597293556624304, 4.21479994225615]


(2.2503864389111965, 1.9644135033449535)

In [12]:
avg_rmse_hor(category_horizon_dict, canada_hor_3_test_dict, 3)

RMSE:  2.3355608750184285
MSE std:  2.0766500086334156
interval: [0.2589108663850128, 4.412210883651844]


(2.3355608750184285, 2.0766500086334156)

In [13]:
avg_rmse_hor(category_horizon_dict, canada_hor_4_test_dict, 4)

RMSE:  2.358562247317593
MSE std:  2.1130892730943662
interval: [0.2454729742232269, 4.471651520411959]


(2.358562247317593, 2.1130892730943662)

In [14]:
avg_rmse_hor(category_horizon_dict, canada_hor_8_test_dict, 8)

RMSE:  2.448978014345627
MSE std:  2.202897234420131
interval: [0.24608077992549582, 4.651875248765758]


(2.448978014345627, 2.202897234420131)

In [15]:
#category_horizon_dict - prediction dict per category per horizon
# us_hor_x_test_dict - actual dict per category for horizon x

In [16]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_canada_2_period_dataset_dict.pickle', 'rb') as f:
    hor_2_raw_dataset_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_canada_3_period_dataset_dict.pickle', 'rb') as f:
    hor_3_raw_dataset_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_canada_4_period_dataset_dict.pickle', 'rb') as f:
    hor_4_raw_dataset_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_canada_8_period_dataset_dict.pickle', 'rb') as f:
    hor_8_raw_dataset_dict = pickle.load(f)

In [17]:
def create_test_dataframe(raw_dataset_dict: dict, horizon:int):
    test_dict = {}
    for key in raw_dataset_dict.keys():
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Inflation t+'+str(horizon)]]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+'+str(horizon): 'Actual Horizon '+str(horizon)}, inplace=True)
        target_df = df[df['Year'] > 2020]
        target_df = target_df.reset_index(drop = True)
        test_dict[key] = target_df
    return test_dict

In [18]:
test_dict_hor_2 = create_test_dataframe(hor_2_raw_dataset_dict, 2)
test_dict_hor_3 = create_test_dataframe(hor_3_raw_dataset_dict, 3)
test_dict_hor_4 = create_test_dataframe(hor_4_raw_dataset_dict, 4)
test_dict_hor_8 = create_test_dataframe(hor_8_raw_dataset_dict, 8)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_4860/3431962343.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_4860/3431962343.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+'+str(horizon): 'Actual Horizon '+str(horizon)}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_4860/3431962343.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [19]:
def get_df_with_predictions_horizon(prediction_dic: dict, actual_dic:dict, horizon: int) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key][horizon].iloc[:,-1:].reset_index(drop = True)
        predictions = predictions.rename(columns = {f'Inflation t+{horizon}': f'Prediction Horizon {horizon}'})
        actuals = actual_dic[key]

        data = pd.concat([predictions, actuals], axis=1).loc[:,['Category','Date','Year',f'Prediction Horizon {horizon}', f'Actual Horizon {horizon}']]
        all_data_dict[key] = data
        
    return all_data_dict

In [20]:
all_data_test_dict_hor_2 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_2,2)
all_data_test_dict_hor_3 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_3,3)
all_data_test_dict_hor_4 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_4,4)
all_data_test_dict_hor_8 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_8,8)

In [21]:
all_data_test_dict_hor_2['All-items']

,Category,Date,Year,Prediction Horizon 2,Actual Horizon 2
0,All-items,2021-01-15,2021.0,0.169375,0.502694
1,All-items,2021-02-15,2021.0,0.338491,0.500180
2,All-items,2021-03-15,2021.0,0.397297,0.497690
3,All-items,2021-04-15,2021.0,0.301117,0.283286
4,All-items,2021-05-15,2021.0,0.132535,0.634475
5,All-items,2021-06-15,2021.0,0.020472,0.210600
6,All-items,2021-07-15,2021.0,0.409235,0.210158
7,All-items,2021-08-15,2021.0,0.195193,0.697353
8,All-items,2021-09-15,2021.0,0.176167,0.208261
9,All-items,2021-10-15,2021.0,0.068259,-0.138793


In [22]:
def plot_results_horizon(all_data_dict, categories, horizon):
    #category_samples = random.sample(categories, 5)+['All items']
    category_samples = categories
    for category in category_samples:
        category_df = all_data_dict[category]
        y_pred = category_df[f'Prediction Horizon {horizon}'].values
        y_actual = category_df[f'Actual Horizon {horizon}'].values

        non_null_indices = ~np.isnan(y_pred) & ~np.isnan(y_actual)

        predicted_values_non_null = y_pred[non_null_indices]
        observed_values_non_null = y_actual[non_null_indices]

        mse = mean_squared_error(predicted_values_non_null, observed_values_non_null)
        print(f'Category is: {category}')
        print(f'RMSE is: {np.sqrt(mse)}')

        fig = px.line(category_df, x="Date", y=[f'Actual Horizon {horizon}', f'Prediction Horizon {horizon}'], title=f'{category} - Actual VS Prediction: horizon {horizon}')
        fig.show()



In [23]:
def total_corr_horizon(all_data_test_dict, horizon):
    corr_dict = {}
    for key in list(all_data_test_dict.keys()):
        df = all_data_test_dict[key]
        y_pred = df[f'Prediction Horizon {horizon}'].values
        y_actual = df[f'Actual Horizon {horizon}'].values

        non_null_indices = ~np.isnan(y_pred) & ~np.isnan(y_actual)

        predicted_values_non_null = y_pred[non_null_indices]
        observed_values_non_null = y_actual[non_null_indices]

        corr = stats.pearsonr(predicted_values_non_null,observed_values_non_null)[0]
        corr_dict[key] =  corr
    
    total_corr = sum(corr_dict.values())
    
    num_high_corr = 0
    for category in corr_dict:
        if corr_dict[category] >= 0.5:
            num_high_corr +=1
    
    print(f'Number of categories with High Correlation: {num_high_corr}')
    print(f'Total Corr Horizon {horizon}, {total_corr}')
    return total_corr

In [24]:
total_corr_horizon(all_data_test_dict_hor_2, 2)
total_corr_horizon(all_data_test_dict_hor_3, 3)
total_corr_horizon(all_data_test_dict_hor_4, 4)
total_corr_horizon(all_data_test_dict_hor_8, 8)

Number of categories with High Correlation: 8
Total Corr Horizon 2, 9.573227277936295
Number of categories with High Correlation: 5
Total Corr Horizon 3, -7.776267069233403
Number of categories with High Correlation: 2
Total Corr Horizon 4, -18.372801386337404
Number of categories with High Correlation: 9
Total Corr Horizon 8, -2.9540526665222577


-2.9540526665222577

In [25]:
cats = list(all_data_test_dict_hor_2.keys())
categories = random.sample(cats, 5)+['All-items']

In [26]:
#categories = list(all_data_test_dict_hor_2.keys())
plot_results_horizon(all_data_test_dict_hor_2, categories, 2)

Category is: Household appliances
RMSE is: 1.6836363826684801


Category is: Upholstered furniture
RMSE is: 2.1346361485871728


Category is: Digital computing equipment and devices
RMSE is: 3.261356108525159


Category is: Other lessons, courses and education services
RMSE is: 0.9842122063544094


Category is: Recreational services
RMSE is: 0.683679762386068


Category is: All-items
RMSE is: 0.5401958079127871


In [27]:
#categories = list(all_data_test_dict_hor_3.keys())
plot_results_horizon(all_data_test_dict_hor_3, categories, 3)

Category is: Household appliances
RMSE is: 1.8437998405228782


Category is: Upholstered furniture
RMSE is: 2.527604911243888


Category is: Digital computing equipment and devices
RMSE is: 3.2190628363035647


Category is: Other lessons, courses and education services
RMSE is: 1.0296306803860595


Category is: Recreational services
RMSE is: 0.7935296068324649


Category is: All-items
RMSE is: 0.6257969369900912


In [28]:
#categories = list(all_data_test_dict_hor_4.keys())
plot_results_horizon(all_data_test_dict_hor_4, categories, 4)

Category is: Household appliances
RMSE is: 1.932138598985306


Category is: Upholstered furniture
RMSE is: 2.384327672636149


Category is: Digital computing equipment and devices
RMSE is: 3.1305278399137264


Category is: Other lessons, courses and education services
RMSE is: 1.005373765115715


Category is: Recreational services
RMSE is: 0.8279851459878972


Category is: All-items
RMSE is: 0.5617335283131089


In [29]:
#categories = list(all_data_test_dict_hor_8.keys())
plot_results_horizon(all_data_test_dict_hor_8, categories, 8)

Category is: Household appliances
RMSE is: 2.200858624826162


Category is: Upholstered furniture
RMSE is: 2.6280971294311994


Category is: Digital computing equipment and devices
RMSE is: 3.272493699685275


Category is: Other lessons, courses and education services
RMSE is: 1.1366751880241357


Category is: Recreational services
RMSE is: 0.7050301696233238


Category is: All-items
RMSE is: 0.5984481749510752
